In [10]:
from games.tic_tac_toe import TicTacToeOpSp as TicTacToe, simple_depth_dependant_scoring_function
from src.game.agents import AIAgentOpSp as AIAgent, User
from algorithms.minimax_openspiel_wrapper import MiniMax
from explainers.alphabeta_explainer import AlphaBetaExplainer
from games.tic_tac_toe.interface.gradio_interface import TicTacToeGradioInterface

In [11]:
# Set the game state translator for consistent representations
from algorithms.minimax_openspiel_wrapper import TreeNode
TreeNode.game_state_translator = lambda cls, opsp_state: TicTacToe.opsp_state_to_action_space(opsp_state)

In [12]:
interface_mode = 'jupyter'

In [13]:
explainer = AlphaBetaExplainer()

opponent = AIAgent(agent_id=1, core=MiniMax(simple_depth_dependant_scoring_function, max_depth=6))
game = TicTacToe(players=[opponent, User(agent_id=0)],
                interface_mode=interface_mode, 
                interface_hyperlink_mode=interface_mode == 'gradio')

if interface_mode == 'gradio': # We need to start the interface externally
    game.explaining_agent = opponent
    interface = TicTacToeGradioInterface(game=game, explainer=explainer)

Simulate few initial moves:

In [14]:
simulate_initial_moves = False
if simulate_initial_moves:
    game.act({'where' : (0,0), 'who': 0})
    game.act({'where' : (1,1), 'who': 1})
    game.act({'where' : (1,0), 'who': 0})


# Play and Explain

In [15]:
if interface_mode == 'jupyter':
    await game.start_game()
else:
    interface.start()

setted root


In [16]:
if game.interface_mode == 'jupyter' and opponent.choice is not None:
    explanation = explainer.explain(opponent.choice, 'the best') # can also modify explanation_depth=4, or print_depth=True
    print(explanation)

In [17]:
if game.interface_mode == 'jupyter' and opponent.choice is not None:
    explainer.frameworks['highlevel'].get_adjective('score').skip_statement = False
    explanation = explainer.explain(opponent.choice, 'the best') # can also modify explanation_depth=4, or print_depth=True
    print(explanation)

Print tree for debugging reasons:

In [18]:
if game.interface_mode == 'jupyter' and opponent.choice is not None:
    opponent.core.print_tree()